## Parallelize existing code with Dask.delayed

In this section we parallelize simple for-loop style code with Dask and Dask.delayed.

This is a simple way to use Dask to parallelize existing codebases or build complex systems.  This will also help us to build intuition for future sections.

### Alternative software

The solutions presented in this section are similar to the following tools:

1.  concurrent.futures
2.  multiprocessing.Pool
3.  Airflow/Luigi

### Simplest possible example

We make some simple functions, inc and add, that sleep for a while to simulate work.  We time running these functions normally.  

In the next section we'll parallelize this code

In [ ]:
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

In [ ]:
# This takes three seconds to run because we call each function sequentially, one after the other
%%time
x = inc(1)
y = inc(2)
z = add(x, y)

### Parallelize with dask.delayed decorator

Those two increment calls *could* be called in parallel.

In this section we call `inc` and `add`, wrapped with `dask.delayed`.  This changes those functions so that they don't run immediately, but instead put those functions and arguments into a task graph.  Now when we run our code this runs immediately, but all it does it create a graph.  We then separately compute the result by calling the `.compute()` method.

In [ ]:
import dask

In [ ]:
# This runs immediately, all it does is build a graph
%%time
x = dask.delayed(inc)(1)
y = dask.delayed(inc)(2)
z = dask.delayed(add)(x, y)

In [ ]:
# This actually runs our computation using a local thread pool
%%time
z.compute()

### What just happened?

The `z` object is a lazy `dask.Delayed` object.  This object holds everything we need to compute the final result.  We can compute the result with `.compute()` as above or we can visualize the result with `.visualize()`.

In [ ]:
z

In [ ]:
z.visualize(rankdir="LR")

### Some questions to consider:

-  Why did we go from 3s to 2s?  Why weren't we able to parallelize down to 1s?
-  What would have happened if the inc and add functions didn't include the `sleep(1)`?  Would Dask still be able to speed up this code?
-  What if we have multiple outputs or also want to get access to x or y?

### Parallelizing for-loop code with control flow

Now we look at a more complex problem where we want to delay some parts of the computation, but not others.  Deciding what to delay is the hard part of using dask.delayed.

We don't always want to wrap everything with `dask.delayed`.  In most cases we will mix immediate execution with delayed execution.  Generally we choose to immediately run (not delay) any code that decides what our computation will be (control flow), especially if it is fast, while we choose to delay functions that don't affect the shape of our computation, but do take a while to run.

In the example below we iterate through a list of inputs and, based on whether the input is even or odd we choose to run either `inc` or `double`.  We collect these results into a list and sum them together.  

-   Which parts of this computation decide what other computations we're going to run?  
    We should not wrap these in dask.delayed because we need to know the answer immediately.
-   Which parts of this computation can we delay?

In [ ]:
def double(x):
    sleep(1)
    return 2 * x

def iseven(x):
    return x % 2 == 0

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
%%time

results = []
for x in data:
    if iseven(x):
        y = double(x)
    else:
        y = inc(x)
    results.append(y)
    
total = sum(results)
total

In [ ]:
%%time

results = []
for x in data:
    if iseven(x):  # even
        y = dask.delayed(double)(x)
    else:          # odd
        y = dask.delayed(inc)(x)

    results.append(y)
    
total = dask.delayed(sum)(results)
total.compute()

In [ ]:
total.visualize()

### Some questions to consider:

-  What are other examples of control flow where we can't use delayed?
-  What would have happened if we had delayed the evaluation of `iseven(x)` in the example above?
-  What are your thoughts on delaying `sum`?  This function was both computational but also very fast to run.

### Pandas exercise

In this exercise we read several CSV files and perform a groupby operation in parallel.  We are given sequential code to do this and parallelize it with Dask.delayed.

### Prep data

First, run this code to prep some data.  You don't need to understand this code.

In [ ]:
import dask.dataframe as dd
from datetime import date, timedelta
import dask.multiprocessing
import os
import shutil

In [ ]:
dirname = os.path.join('data', 'stocks')
if os.path.exists(dirname):
    shutil.rmtree(dirname)

os.mkdir(dirname)

def name_function(i):
    return str(date(2015, 1, 1) + timedelta(days=1) * i)

for symbol in ['GOOG', 'YHOO', 'AAPL', 'MSFT']:
    df = dd.demo.daily_stock(symbol, '2015', '2016', freq='1s')
    dirname = os.path.join('data', 'stocks', symbol)
    os.mkdir(dirname)
    df.to_csv(os.path.join('data', 'stocks', symbol, '*.csv'), 
              name_function=name_function, 
              get=dask.multiprocessing.get)

### Inspect data

In [ ]:
os.listdir(os.path.join('data', 'stocks'))

In [ ]:
os.listdir(os.path.join('data', 'stocks', 'GOOG'))

### Read one file with pandas.read_csv and compute spread

In [ ]:
import pandas as pd
df = pd.read_csv(os.path.join('data', 'stocks', 'GOOG', '2015-01-01.csv'), 
                 parse_dates=['timestamp'], 
                 index_col='timestamp')
df.head()

In [ ]:
spread = df.high.max() - df.low.min()
spread

### Plot spread over time

In [ ]:
%%time

from glob import glob
filenames = sorted(glob(os.path.join('data', 'stocks', 'GOOG', '*.csv')))

spreads = []
days = []
for fn in filenames:
    df = pd.read_csv(fn, parse_dates=['timestamp'], index_col='timestamp')
    spread = df.high.max() - df.low.min()
    day = df.index[0].date()
    
    spreads.append(spread)
    days.append(day)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(days, spreads)

### Exercise: parallelize the code above

Use dask.delayed to parallelize the code above.  Some extra things you will need to know.

1.  Methods and attribute access on delayed objects work automatically, so if you have a delayed object you can perform normal arithmetic, slicing, and method calls on it and it will produce the correct delayed calls.

    ```python
    x = delayed(np.arange)(10)
    y = (x + 1)[::2].sum()  # everything here was delayed
    ```
2.  Calling the `.compute()` method works well when you have a single output.  When you have multiple outputs you might want to use the `dask.compute` function:

    ```python
    >>> x = dask.delayed(np.arange)(10)
    >>> y = x ** 2
    >>> min, max = dask.compute(y.min(), y.max())
    (0, 81)
    ```
    
    This way dask can share the intermediate values (like `y = x**2`)
    
So your goal is to parallelize the code above (which has been copied below) using Dask.delayed.  You may also want to visualize a bit of the computation to see if you're doing it correctly.

*Note: performance will improve a little bit, but not a whole lot.  We'll discuss why afterwards*

In [ ]:
%%time

from glob import glob
filenames = sorted(glob(os.path.join('data', 'stocks', 'GOOG', '*.csv')))

spreads = []
days = []
for fn in filenames:
    df = dask.delayed(pd.read_csv)(fn, parse_dates=['timestamp'], index_col='timestamp')
    spread = df.high.max() - df.low.min()
    day = df.index[0].date()
    
    spreads.append(spread)
    days.append(day)
    
spreads, days = dask.compute(spreads, days)